In [1]:
import sqlite3
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import load_model

2024-07-01 21:40:59.340394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-07-01 21:41:00.170351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the model
model = load_model('updated_electricity_price_model.h5')

2024-07-01 21:41:00.871981: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-01 21:41:00.918653: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-01 21:41:00.918922: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect("../prisma/db.sqlite")
# Query recent new feature data
query = "SELECT * FROM HistoricalElectricityWeather WHERE price IS NULL "
data = pd.read_sql_query(query, conn)
data.shape

(314, 11)

In [4]:
data.head()

,id,dateId,price,temperature,precipitation,cloudCover,windSpeed10m,shortwaveRadiation,weatherCodeId,createdDate,modifiedDate
0,2045,2045,None,-10.8850,0.0,19.500002,10.195057,0,0,2024-06-13 10:55:45,2024-06-13 10:55:45
1,10781,10781,None,-15.8350,0.0,0.000000,7.244860,0,0,2024-06-13 10:55:45,2024-06-13 10:55:45
2,19685,19685,None,-0.7350,0.0,92.699997,6.409617,1,3,2024-06-13 10:55:45,2024-06-13 10:55:45
3,21482,21482,None,7.7475,0.0,99.000000,1.800000,8,3,2024-06-13 10:55:45,2024-06-13 10:55:45
4,21483,21483,None,8.0475,0.0,97.000000,1.440000,41,3,2024-06-13 10:55:45,2024-06-13 10:55:45


In [5]:
# Select specific columns using .filter
X = data.filter(['temperature', 'precipitation', 'weatherCodeId', 'cloudCover', 'windSpeed10m', 'shortwaveRadiation'])

In [6]:

# Load the scalers from disk
scaler_X = joblib.load('scaler_X.pkl')
scaler_y = joblib.load('scaler_y.pkl')

/home/ashagaire/Desktop/LearnIT and coding with Finnish Language/Mittwatt/electricity-shoc/mittwatt/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Normalize the features
X_normalized = scaler_X.transform(X)
# Predict using the trained model
y_pred_scaled = model.predict(X_normalized)
# Inverse transform the predicted values to get them back to the original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_pred_flatten = y_pred.flatten()
y_pred_one_d = y_pred_flatten.astype(float).round(2)

I0000 00:00:1719859261.385594   13131 service.cc:145] XLA service 0x72aa8c004570 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719859261.385651   13131 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-07-01 21:41:01.401055: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


2024-07-01 21:41:01.480207: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/10 ━━━━━━━━━━━━━━━━━━━━ 3s 432ms/step

I0000 00:00:1719859261.736151   13131 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step 

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step


In [8]:
# Create the new DataFrame
prediction_dataframe_new = pd.DataFrame({
    'dateId': data['dateId'],
    'price': y_pred_one_d
})

In [9]:
# Insert or update rows
for index, row in prediction_dataframe_new.iterrows():
    date = row['dateId']
    price = row['price']
    insert_or_replace_query = """
    INSERT OR REPLACE INTO ForecastElectricityPrice (dateId, price)
    VALUES (?, ?);
    """
    conn.execute(insert_or_replace_query, (date, price))


In [10]:
# Close the connection
conn.commit()
conn.close()